In [1]:
import time
from joblib import Parallel, delayed
from scipy.io import loadmat 
import numpy as np
from pandas import read_csv
from multiprocessing import Process, Queue
from math import floor


In [4]:

# filename = 'Mug128_singlepixcam.mat'
filename = 'Ball64_singlepixcam.mat'
array = loadmat(filename)

X = np.array(array['A'])
Y = np.array(array['y'])
nr,nc = X.shape
# print X.shape
beta = 1;  # For Squared Loss
lamda = 0.216;

#Normalization-1
minimum = X.min(0);
maximum = X.max(0);

for i in np.arange(nc):
    if(maximum[i]!=minimum[i]):
        X[:,i] = (X[:,i]-minimum[i]) / (maximum[i]-minimum[i]);
    else:
        X[:,i] = np.ones(nr)

#Normalization-2
X = X - X.mean(axis=0)
nrm = np.linalg.norm(X,axis = 0);

for i in np.arange(nc):
	if nrm[i]>0:
		X[:,i] = X[:,i]/nrm[i]

Data = X
X = np.concatenate((X,-X), axis = 1)
nc = 2*nc;
# print nc,X.shape
# print nc
# sys.exit(0)

w = np.zeros(nc,dtype = np.float64).reshape(nc,1) +1
# w = np.random.rand(nc,1)

def func(index):
	# print index
	t1 = np.dot(X,w)
	t2 = t1- Y

	g = np.dot(X[:,index].T,t2) + lamda; #why nr??
	w[index] = w[index] -   0.001*g/beta; # alpha
	# print w[index]
	if w[index]<0:
		w[index] = 0;

	return w[index]
    
for i in np.arange(500):
    for k in np.arange(0,nc-100,100):
        with Parallel(n_jobs=20) as parallel:
            results = parallel(delayed(func)(j)for j in np.arange(k,k+100) )
        w[k:k+100] = results
    
    print (i)    
    temp = nc/2
    z1 = np.subtract(w[temp:nc] , w[0:temp])
    z1 = z1.reshape(temp,1)

    z2 = np.subtract(w[0:temp],w[temp:nc])
    z2 = z2.reshape(temp,1)
    # pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
    pred = np.dot(Data,z1)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z1 w2-w1 MSE Error:%f' % MSE)

    pred = np.dot(Data,z2)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z2  w1-w2  MSE Error:%f' % MSE)
            
    print (np.linalg.norm(w))



0
z1 w2-w1 MSE Error:0.099389
z2  w1-w2  MSE Error:0.099387
90.4903286234


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:65: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1
z1 w2-w1 MSE Error:0.099390
z2  w1-w2  MSE Error:0.099386
90.470989507
2
z1 w2-w1 MSE Error:0.099391
z2  w1-w2  MSE Error:0.099386
90.4516506575
3
z1 w2-w1 MSE Error:0.099391
z2  w1-w2  MSE Error:0.099385
90.4323120578
4
z1 w2-w1 MSE Error:0.099392
z2  w1-w2  MSE Error:0.099384
90.412973742
5
z1 w2-w1 MSE Error:0.099393
z2  w1-w2  MSE Error:0.099383
90.3936356711
6
z1 w2-w1 MSE Error:0.099394
z2  w1-w2  MSE Error:0.099383
90.3742978655
7
z1 w2-w1 MSE Error:0.099395
z2  w1-w2  MSE Error:0.099382
90.3549603235
8
z1 w2-w1 MSE Error:0.099396
z2  w1-w2  MSE Error:0.099381
90.3356230281
9
z1 w2-w1 MSE Error:0.099396
z2  w1-w2  MSE Error:0.099381
90.3162859838
10
z1 w2-w1 MSE Error:0.099397
z2  w1-w2  MSE Error:0.099380
90.2969492101
11
z1 w2-w1 MSE Error:0.099398
z2  w1-w2  MSE Error:0.099380
90.2776126907
12
z1 w2-w1 MSE Error:0.099399
z2  w1-w2  MSE Error:0.099379
90.2582764251
13
z1 w2-w1 MSE Error:0.099400
z2  w1-w2  MSE Error:0.099378
90.2389404122
14
z1 w2-w1 MSE Error:0.099401
z2  w

Process ForkPoolWorker-746952:
Process ForkPoolWorker-746944:
Process ForkPoolWorker-746948:
Process ForkPoolWorker-746957:
Traceback (most recent call last):
Process ForkPoolWorker-746942:
Process ForkPoolWorker-746947:
Process ForkPoolWorker-746958:
Process ForkPoolWorker-746941:
  File "/home/clabuser/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-746945:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/clabuser/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/clabuser/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/clabuser/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-74695

KeyboardInterrupt: 

In [3]:
temp = nc/2
z = w[temp:nc] - w[0:temp]
z = z.reshape(temp,1)
# pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
pred = np.dot(Data,z)
differ = (pred-Y)
MSE = np.mean( np.square(differ) );
print ('MSE Error:%f' % MSE)

MSE Error:0.099396


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
